In [1]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.core.composition import OneOf
import glob
import os
import numpy as np

In [2]:
df = pd.read_csv('/workspace/#Ub370#Uc774#Ud130/Data/label/train/train.csv')
df.head()

,filename,maxX,maxY,minX,minY,faceExp_uploader,path
0,6oj439e3fbcc52759fb3093035b7c0ecc55c93543dae63...,2073.368938,1488.055775,1339.858438,531.096642,분노,/workspace/#Ub370#Uc774#Ud130/Data/img/train/a...
1,65rsfe402042f34319e10128c1ab9614e2f967690a64a0...,2889.123830,2305.965033,1804.280169,695.748561,분노,/workspace/#Ub370#Uc774#Ud130/Data/img/train/a...
2,b1cbe34734870cc11c33334e02bea93ac3a3b061caab62...,1979.987544,1569.049740,1055.479759,463.398892,분노,/workspace/#Ub370#Uc774#Ud130/Data/img/train/a...
3,llfycc0aa29599cc63cace3610fdaaad3a99aab2ee38c9...,855.695310,1162.188686,150.693090,298.234386,분노,/workspace/#Ub370#Uc774#Ud130/Data/img/train/a...
4,3hww73b70615461a7336d0383b53582f8bf804f6e0f30d...,1403.826125,1385.677361,617.136999,297.420141,분노,/workspace/#Ub370#Uc774#Ud130/Data/img/train/a...


In [3]:
df['faceExp_uploader'].value_counts()

faceExp_uploader
분노    1500
당황    1500
슬픔    1499
기쁨    1494
Name: count, dtype: int64

In [4]:

# .npz 파일 로드
npz_train_anger = np.load('/workspace/#Ub370#Uc774#Ud130/Data/segmentation/train/train_anger.npz')
npz_train_happy = np.load('/workspace/#Ub370#Uc774#Ud130/Data/segmentation/train/train_happy_updated.npz')
npz_train_panic = np.load('/workspace/#Ub370#Uc774#Ud130/Data/segmentation/train/train_panic_updated.npz')
npz_train_sadness = np.load('/workspace/#Ub370#Uc774#Ud130/Data/segmentation/train/train_sadness.npz')


In [5]:
processed_data = []

for index, row in df.iterrows():
    # if index == 5:
    #     break
    npz_train = npz_train_anger
    if row['faceExp_uploader'] == '기쁨':
        npz_train = npz_train_happy
    elif row['faceExp_uploader'] == '당황':
        npz_train = npz_train_panic
    elif row['faceExp_uploader'] == '슬픔':
        npz_train = npz_train_sadness

    data = npz_train[row['filename']]  # Segmentation 데이터 로드

    image = cv2.imread(row['path'] + row['filename'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 원본 이미지의 크기 (width, height)
    original_height, original_width = image.shape[:2]

    # 새로운 이미지 생성 (검정색 초기화)
    new_img = image.copy()
    new_img[:, :] = [0, 0, 0]  # 검정색으로 초기화
    mask = data == 3  # 얼굴 영역 추출
    new_img[mask] = image[mask]  # 얼굴만 원본 이미지에서 가져와 새로운 이미지에 덮어씌움

    # 얼굴만 남긴 이미지를 640x640으로 리사이즈 후 저장
    output_file_path = os.path.join('/workspace/Data_Augumentation/segmentation_face/image/train/anger/', f"face_only_{row['filename']}")
    if row['faceExp_uploader'] == '기쁨':
        output_file_path = os.path.join('/workspace/Data_Augumentation/segmentation_face/image/train/happy/', f"face_only_{row['filename']}")
    elif row['faceExp_uploader'] == '당황':
        output_file_path = os.path.join('/workspace/Data_Augumentation/segmentation_face/image/train/panic/', f"face_only_{row['filename']}")
    elif row['faceExp_uploader'] == '슬픔':
        output_file_path = os.path.join('/workspace/Data_Augumentation/segmentation_face/image/train/sadness/', f"face_only_{row['filename']}")

    new_img_bgr = cv2.cvtColor(new_img, cv2.COLOR_RGB2BGR)  # OpenCV는 BGR 포맷 사용
    resized_img = cv2.resize(new_img_bgr, (640, 640))  # 리사이즈
    cv2.imwrite(output_file_path, resized_img)  # 이미지 저장

    # 바운딩 박스 크기 비율 계산
    width_ratio = 640 / original_width
    height_ratio = 640 / original_height

    new_x_min = int(row['minX'] * width_ratio)
    new_y_min = int(row['minY'] * height_ratio)
    new_x_max = int(row['maxX'] * width_ratio)
    new_y_max = int(row['maxY'] * height_ratio)

    path = '/workspace/Data_Augumentation/segmentation_face/image/train/anger/'
    if row['faceExp_uploader'] == '기쁨':
        path = '/workspace/Data_Augumentation/segmentation_face/image/train/happy/'
    elif row['faceExp_uploader'] == '당황':
        path = '/workspace/Data_Augumentation/segmentation_face/image/train/panic/'
    elif row['faceExp_uploader'] == '슬픔':
        path = '/workspace/Data_Augumentation/segmentation_face/image/train/sadness/'

    # 변경된 데이터 저장
    processed_data.append({
        'filename': f"face_only_{row['filename']}",
        'maxX': new_x_max,
        'maxY': new_y_max,
        'minX': new_x_min,
        'minY': new_y_min,
        'faceExp_uploader': row['faceExp_uploader'],
        'path': path
    })

output_csv_path = '/workspace/Data_Augumentation/segmentation_face/label/train/face_only_train.csv'

# 처리된 데이터 저장
if processed_data:
    processed_df = pd.DataFrame(processed_data)
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    processed_df.to_csv(output_csv_path, index=False)
    print(f"Processed CSV saved to: {output_csv_path}")
else:
    print("No data processed.")
    


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS

Processed CSV saved to: /workspace/Data_Augumentation/segmentation_face/label/train/face_only_train.csv
